In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
filepath = "../1. Data Scraping/output/data.json"

In [3]:
# Load JSON data
with open(filepath, "r") as file:
    data = json.load(file)

# Extracting data
rows = []
for date, details in data.items():
    x = details["aqi"]
    if( x != None ):
        if( x["value"] != -1):
            row = {"Date": date}  # Daily date
            for metric in details["metrics"]:

                row[metric["name"]] = metric["avg"]  # Daily avg pollutant values

            row["AQI"] = x["value"]
    rows.append(row)


# Convert to DataFrame
df = pd.DataFrame(rows)

# Save as CSV
# df.to_csv("AQI_data.csv", index=False)

# Display the first few rows
df.head()


,Date,PM2.5,PM10,NO2,NH3,SO2,CO,OZONE,AQI
0,2019-05-01T00:00:00Z,81,140,13,2,28,22,23,140
1,2019-05-02T00:00:00Z,129,214,14,2,17,25,25,214
2,2019-05-03T00:00:00Z,89,144,11,2,17,13,43,144
3,2019-05-04T00:00:00Z,-,-,-,-,-,20,23,23
4,2019-05-05T00:00:00Z,78,116,6,2,13,27,25,116


In [23]:
# Load the CSV file
df = pd.read_csv('AQI_data.csv')

# Replace '-' with NaN for proper numerical handling
df.replace('-', np.nan, inplace=True)

df.replace(' ', np.nan, inplace=True)

# Save as CSV
# df.to_csv("AQI_data.csv", index=False)

df.head()

,Date,PM2.5,PM10,NO2,NH3,SO2,CO,OZONE,AQI
0,2019-05-01T00:00:00Z,81,140,13,2,28,22,23,140
1,2019-05-02T00:00:00Z,129,214,14,2,17,25,25,214
2,2019-05-03T00:00:00Z,89,144,11,2,17,13,43,144
3,2019-05-04T00:00:00Z,NaN,NaN,NaN,NaN,NaN,20,23,23
4,2019-05-05T00:00:00Z,78,116,6,2,13,27,25,116


In [20]:

# Convert pollutant columns to numeric
df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric)

def impute_missing_values(df, window_size=5):
    for col in df.columns[1:-1]:  # Skip 'Date' and 'AQI'
        df[col] = df[col].fillna(df[col].rolling(window=window_size, min_periods=1).mean())
    return df


# Impute missing pollutant values
df = impute_missing_values(df)

# Update the AQI column (max of pollutants per day)
df['AQI'] = df.iloc[:, 1:-1].max(axis=1)

# Save the cleaned data
# df.to_csv('AQI_data_cleaned_1.csv', index=False)

# Display the first few rows of the cleaned dataset
df.head()

/var/folders/8h/lm8d9r6538qdgrp61j_6zzlr0000gn/T/ipykernel_53332/991363776.py:14: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(df[col].rolling(window=window_size, min_periods=1).mean())


,Date,PM2.5,PM10,NO2,NH3,SO2,CO,OZONE,AQI
0,2019-05-01T00:00:00Z,81.000000,140.0,13.000000,2.0,28.000000,22.0,23.0,140.0
1,2019-05-02T00:00:00Z,129.000000,214.0,14.000000,2.0,17.000000,25.0,25.0,214.0
2,2019-05-03T00:00:00Z,89.000000,144.0,11.000000,2.0,17.000000,13.0,43.0,144.0
3,2019-05-04T00:00:00Z,99.666667,166.0,12.666667,2.0,20.666667,20.0,23.0,166.0
4,2019-05-05T00:00:00Z,78.000000,116.0,6.000000,2.0,13.000000,27.0,25.0,116.0
